# Imports

In [3]:
import requests
import os
import sys
import web3
from eth_account import Account
from eth_account.signers.local import LocalAccount
from web3 import Web3, EthereumTesterProvider
from web3.middleware import construct_sign_and_send_raw_middleware

import solcx

# Functions

In [4]:
def compile_source_file(file_path):
    with open(file_path, 'r') as f:
        source = f.read()
    return solcx.compile_source(source, output_values=["abi", "bin", "bin-runtime"])

# Deploy vesting contract to Goerli

In [5]:
# Get the Infura key.
infura_key=os.environ["INFURA_KEY"]
# Connect with Infura.
infura_url = f"https://goerli.infura.io/v3/{infura_key}"

w3 = web3.Web3(web3.Web3.HTTPProvider(infura_url))
print("isConnected=", w3.isConnected())

isConnected= True


In [6]:
account_addr = "0x6531e2613bbbEEcd898356F7b9bEfBaEfd42804B"
private_key = os.environ["PRIVATE_KEY"]

account: LocalAccount = Account.from_key(private_key)
w3.middleware_onion.add(construct_sign_and_send_raw_middleware(account))

In [21]:
compiled_vesting = compile_source_file("contracts/TokenTimelock.sol")
interface_vesting = compiled_vesting["<stdin>:TokenTimelock"]

In [8]:
token_addr = "0x14E3Cd22bAF93aAE4f8F576fD8Fb918f1016bB81"
func = w3.eth.contract(
    abi=interface["abi"],
    bytecode=interface["bin"]).constructor(token_addr)
# Deploy the Vesting to Goerli.
tx_hash = func.transact({"from": account_addr})

In [9]:
tx_hash

HexBytes('0x08e2dfa9c1c754993621e094284800bb2f4b34ceb588a9309321e16e721c17aa')

In [10]:
reciept = w3.eth.get_transaction_receipt(tx_hash)

In [11]:
reciept

AttributeDict({'blockHash': HexBytes('0x7ad95146cadd45bbe1401a709496e128a06c08cc2c61fc9fdcbeacd922fb5863'),
 'blockNumber': 8577860,
 'contractAddress': '0x9727Bf5A74efDfB346ee6b47086bE4f52d7D422a',
 'cumulativeGasUsed': 21209946,
 'effectiveGasPrice': 135673366166,
 'from': '0x6531e2613bbbEEcd898356F7b9bEfBaEfd42804B',
 'gasUsed': 1620365,
 'logs': [AttributeDict({'address': '0x9727Bf5A74efDfB346ee6b47086bE4f52d7D422a',
   'blockHash': HexBytes('0x7ad95146cadd45bbe1401a709496e128a06c08cc2c61fc9fdcbeacd922fb5863'),
   'blockNumber': 8577860,
   'data': '0x',
   'logIndex': 188,
   'removed': False,
   'topics': [HexBytes('0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
    HexBytes('0x0000000000000000000000006531e2613bbbeecd898356f7b9befbaefd42804b')],
   'transactionHash': HexBytes('0x08e2dfa9c1c754993621e094284800bb2f4b34ceb588a9309321e16e721c17aa'),
   'transactionIndex': 120})]

# Deployment price

In [12]:
gas_used = 1_620_365
gas_price_gwei = 23
eth_price = 1_654

total_gwei = gas_used * gas_price_gwei
total_eth = total_gwei * 0.000000001

In [14]:
total_eth

0.037268395

In [13]:
total_eth * eth_price

61.64192533000001

# Interactions

Get vesting contract.

In [22]:
vesting = w3.eth.contract(address=reciept["contractAddress"], abi=interface_vesting["abi"])

Send Sorrentum tokens to the vesting contract

In [20]:
# Get token contract.
compiled_token = compile_source_file("contracts/SorrentumToken.sol")
interface_token = compiled_token["<stdin>:SorrentumToken"]
surr_token = w3.eth.contract(address=token_addr, abi=interface["abi"])

Send 10 tokens to vesting contract.

In [45]:
tx_hash = surr_token.functions.transfer(reciept["contractAddress"], 10*10**18).transact({"from": account_addr})

Check the balance.

In [46]:
vesting.functions.getContractBalance().call()

1000000000000000000

**Add student 1**

In [26]:
student1 = "0x051D41a881dcc9E946Cd08aE030f3Baf31d12550"
# Wed Mar 01 2023 13:10:24 GMT+0000
release_time1 = 1677676224

In [27]:
tx_hash = vesting.functions.addParticipant(student1, 3*10**18, release_time1).transact({"from": account_addr})

In [28]:
tx_hash

HexBytes('0xec5001605864a6cd300f9c9e8111509b5422823980befabd3b03a18c978d3ad7')

Check the student data.

In [33]:
vesting.functions.getReleaseTimestamp(student1).call()

1677676224

In [34]:
vesting.functions.getTokensAmountLocked(student1).call() 

3000000000000000000

**Add student 2**

In [30]:
student2 = "0xad1Ef7f653995b55C7B4005B9bE18Ab989F7128e"
# Wed Mar 01 2023 13:30:24 GMT+0000
release_time2 = 1677677424

In [31]:
tx_hash = vesting.functions.addParticipant(student2, 2*10**18, release_time2).transact({"from": account_addr})

In [32]:
tx_hash

HexBytes('0xc347035e87b487a43db5c2a51e969e60a7b7aa2bd85d45e7183cc6ac2bc49454')

Check student data

In [35]:
vesting.functions.getReleaseTimestamp(student2).call()

1677677424

In [36]:
vesting.functions.getTokensAmountLocked(student2).call() 

2000000000000000000

**Add 2 more tokens to student 2**

In [37]:
tx_hash = vesting.functions.addAmount(student2, 2*10**18).transact({"from": account_addr})

In [38]:
tx_hash

HexBytes('0x566005b1fb276e710dbbc57a6a45ce947c01907e1af5aefeeafa1d99e2f6e481')

Check updated student amount.

In [39]:
vesting.functions.getTokensAmountLocked(student2).call() 

4000000000000000000

**Release tokens**

Connect to student1 wallet.

In [55]:
student1_private_key = os.environ["STUDENT1_KEY"]

account: LocalAccount = Account.from_key(student1_private_key)
w3.middleware_onion.add(construct_sign_and_send_raw_middleware(account))

Check student balance.

In [42]:
surr_token.functions.balanceOf(student1).call()

0

Release tokens (student1 account needs to have some ETH to make a transaction)

In [58]:
tx_hash = vesting.functions.releaseTokens().transact({"from": student1})

In [59]:
tx_hash

HexBytes('0x29b48db72685565e7d711676fa9bf7eedf6f9631fa4aa5290129482d50eed65d')

In [60]:
surr_token.functions.balanceOf(student1).call()

3000000000000000000